In [1]:
!pip install pandas plotly matplotlib seaborn scikit-learn numpy --upgrade --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.3.1 which is incompatible.
ydata-profiling 4.12.2 requires numpy<2.2,>=1.16.0, but you have numpy 2.3.1 which is incompatible.


In [2]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [43]:
data=pd.DataFrame(pd.read_csv('underwater_temperature.csv',encoding='utf-8'))
data

,ID,Site,Latitude,Longitude,Date,Time,Temp (*C),Depth
0,1,Ilha Deserta,27.2706,48.3310,2013/02/20,11:40:02,24.448,12.0
1,2,Ilha Deserta,27.2706,48.3310,2013/02/20,12:00:03,24.448,12.0
2,3,Ilha Deserta,27.2706,48.3310,2013/02/20,12:20:04,24.545,12.0
3,4,Ilha Deserta,27.2706,48.3310,2013/02/20,12:40:05,24.448,12.0
4,5,Ilha Deserta,27.2706,48.3310,2013/02/20,13:00:06,24.351,12.0
...,...,...,...,...,...,...,...,...
408633,408634,lha do Xavier,27.6092,48.3858,2013/03/07,09:03:29,24.931,5.0
408634,408635,lha do Xavier,27.6092,48.3858,2013/03/07,09:23:29,24.931,5.0
408635,408636,lha do Xavier,27.6092,48.3858,2013/03/07,09:43:29,24.931,5.0
408636,408637,lha do Xavier,27.6092,48.3858,2013/03/07,10:03:29,24.931,5.0


In [44]:
data=data.dropna()

In [45]:
year=pd.to_datetime(data['Date']).dt.year

In [46]:
X=data[year<2014]
X=X.drop(['Date','Time','ID'],axis=1)
X_inp=X[['Site','Latitude','Longitude','Depth']]
X_tar=X['Temp (*C)']

Y=data[year>=2014]
Y=Y.drop(['Date','Time','ID'],axis=1)
Y_inp=Y[['Site','Latitude','Longitude','Depth']]
Y_tar=Y['Temp (*C)']

obj=X_inp.select_dtypes(include='object').columns
num=X_inp.select_dtypes(exclude='object').columns
X_tar

0         24.448
1         24.448
2         24.545
3         24.448
4         24.351
           ...  
408633    24.931
408634    24.931
408635    24.931
408636    24.931
408637    24.931
Name: Temp (*C), Length: 263029, dtype: float64

In [47]:
num_trans=SimpleImputer(strategy='constant')
cat_trans=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer([
    ('num',num_trans,num),
    ('cat',cat_trans,obj)
])
model=LinearRegression()
pipe=Pipeline([
    ('preprocessor',preprocessor),
    ('model',model)
])
pipe.fit(X_inp,X_tar)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
p=pipe.predict(Y_inp)
mae=mean_absolute_error(Y_tar,p)
mae/np.median(Y_tar)

array([20.60302536, 20.60302536, 20.60302536, ..., 20.25250411,
       20.25250411, 20.25250411], shape=(145605,))